In [6]:
import pandas as pd
import numpy as np

import requests

from datetime import datetime, timedelta
from pprint import pprint



### EJEMPLO


In [7]:
lang    =  "es"
category = "balance"
widget   = "balance-electrico"
query   = "start_date=2018-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=year"


In [8]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [9]:
endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
response = requests.get(url = endpoint, headers = headers)
data = response.json()
data

{'data': {'type': 'Balance de energía eléctrica',
  'id': 'bal1',
  'attributes': {'title': 'Balance de energía eléctrica',
   'last-update': '2019-06-12T17:40:40.000+02:00',
   'description': 'Balance eléctrico: asignación de unidades de producción según combustible principal. La producción neta de las instalaciones no renovables e hidráulicas UGH tiene descontados los consumos propios de las instalaciones. En dichos tipos de producción una generación negativa indica que la electricidad horaria consumida para los usos de las plantas excede la producción horaria de las instalaciones.'},
  'meta': {'cache-control': {'cache': 'HIT',
    'expireAt': '2024-10-26T09:03:45'}}},
 'included': [{'type': 'Renovable',
   'id': 'Renovable',
   'attributes': {'title': 'Renovable',
    'last-update': '2019-06-12T17:40:36.000+02:00',
    'description': None,
    'magnitude': None,
    'content': [{'type': 'Hidráulica',
      'id': '10288',
      'groupId': 'Renovable',
      'attributes': {'title': '

# OBTENCIÓN DE DATOS DE BALANCE




In [10]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [11]:
def balance_datos(lang, category, widget, restaDia, input_año):

    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')


    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

    query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"



    endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()



    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()


    for dato in data['included']: 
        for info in dato['attributes']['content']:
            nombre = info['attributes']['title']

            tipo = info['groupId']


        for i in info['attributes']['values']:
            valor = i['value']

            porcentaje = i['percentage']

            fecha = i['datetime']
            fecha = pd.to_datetime(fecha)
            fecha = fecha.strftime('%d/%m/%Y')
            
            
            lista_nombres.append(nombre)
            lista_tipos.append(tipo)
            lista_valores.append(valor)
            lista_porcentajes.append(porcentaje)
            lista_fechas.append(fecha)

    df_balance = pd.DataFrame()
    df_balance['nombre']               = lista_nombres
    df_balance['tipo de energía']      = lista_tipos
    df_balance['Valores']              = lista_valores
    df_balance["Porcentaje"]           = lista_porcentajes
    df_balance["Fecha actualización"]  = lista_fechas               

    return df_balance  


In [12]:
lang    =  input('¿En español (es) o en inglés (en)?') 
category = input('¿Balance, demanda, generación o intercambio?') 
widget   = input('¿Qué widget quieres?')


restaDia = float(input("¿Cuantos días atrás?: ______________"))
input_año = int(input("¿Qué año?: ______________"))

balance_datos(lang, category, widget, restaDia, input_año)

,nombre,tipo de energía,Valores,Porcentaje,Fecha actualización
0,Renewable generation,Renewable,283293.8550,1,28/07/2022
1,Renewable generation,Renewable,315791.7010,1,29/07/2022
2,Renewable generation,Renewable,358667.5460,1,30/07/2022
3,Renewable generation,Renewable,312348.7135,1,31/07/2022
4,Renewable generation,Renewable,306444.6200,1,01/08/2022
...,...,...,...,...,...
178,Demand at busbars,Demand at busbars,695597.3890,1,22/09/2022
179,Demand at busbars,Demand at busbars,689988.5230,1,23/09/2022
180,Demand at busbars,Demand at busbars,602468.7460,1,24/09/2022
181,Demand at busbars,Demand at busbars,547492.9340,1,25/09/2022


In [14]:
balance_datos(lang, category, widget, restaDia, input_año).to_csv('balance_electrico.csv')